In [1]:
!pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\tuanq\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\tuanq\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import findspark
findspark.init("C:\spark-3.5.5-bin-hadoop3")
import pyspark
from pyspark.sql import SparkSession

scala_version = "2.12"
spark_version = "3.5.5"
packages = [
    f"org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}",
    "org.apache.kafka:kafka-clients:3.6.0"
]

# spark = SparkSession.builder \
#     .master("local") \
#     .appName("kafka-example") \ 
#     .config("spark.jars.packages", ",".join(packages)) \
#     .getOrCreate()
    
spark = SparkSession.builder \
    .appName("kafka-example") \
    .master("local") \
    .config("spark.jars.packages", ",".join(packages)) \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true") \
    .config("spark.hadoop.io.nativeio.enabled", "false") \
    .config("spark.sql.broadcastTimeout", "600") \
    .getOrCreate()

In [6]:
!pip install pyarrow



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\tuanq\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from pyspark.sql.functions import get_json_object, col, udf, pandas_udf
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
from kafka import KafkaConsumer
import json
import pandas as pd
from time import sleep
from IPython.display import display, clear_output

# Setup model
model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True, truncation=True, max_length=512)

# # Predict function: returns top label or multiple labels
# def predict_top_label(text, threshold=0.6)-> str:
#     scores = classifier(text)[0]
#     toxic_labels = [s for s in scores if s['score'] >= threshold]
#     if toxic_labels:
#         return toxic_labels[0]['label']  # Or return all if you want multi-label
#     return 'not_toxic'

# # Wrap it as a Spark UDF
# predict_udf = udf(predict_top_label, StringType())

@pandas_udf(StringType())
def predict_batch_udf(texts: pd.Series) -> pd.Series:
    # Load once per worker
    if not hasattr(predict_batch_udf, "classifier"):
        model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
        tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
        predict_batch_udf.classifier = pipeline(
            "text-classification",
            model=model,
            tokenizer=tokenizer,
            return_all_scores=True
        )

    def predict_label(text: str) -> str:
        try:
            scores = predict_batch_udf.classifier(text)[0]
            for s in scores:
                if s["score"] >= 0.6:
                    return s["label"]
            return "not_toxic"
        except Exception:
            return "error"

    return texts.apply(predict_label)

topic_name = 'youtube-comments-5'
kafka_server= 'localhost:9092'

schema = StructType([
    StructField("comment_id", StringType()),
    StructField("author", StringType()),
    StructField("text", StringType()),
    StructField("published_at", StringType()),
    StructField("like_count", IntegerType())
])

kafkaDf = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_server) \
        .option("subscribe", topic_name) \
        .option("startingOffsets", "latest") \
        .load()

kafkaDf2 = kafkaDf.selectExpr(
    "CAST(key AS STRING) as key",
    "CAST(value AS STRING) as value",
    "CAST(timestamp AS STRING) as timestamp"
)

parsedDF = kafkaDf2.select(get_json_object(col("value"), "$.comment").alias("comment"))

predictedDf = parsedDF.withColumn("predicted_label", predict_batch_udf(col("comment")))

# from random import randint
# randNum=str(randint(0,10000))
qname = "predictions"

# Output to console (for debugging)
query = (predictedDf.writeStream \
    .queryName(qname) \
    .format("memory") \
    .outputMode("append") \
    # .option("checkpointLocation", "file:///C:/tmp/spark_ckpt_yt") \
    .trigger(processingTime="3 seconds") \
    .start())
    # .option("kafka.bootstrap.servers", kafka_server) \
    # .option("topic", topic_name) \
    # .option("checkpointLocation", "C:/tmp/spark_checkpoints") \
    # .option("startingOffsets", "latest") \

# query.awaitTermination()

# print("Active?", query.isActive)
try:
    while True:
        df = spark.sql("SELECT * FROM predictions")
        display(df.show())
        sleep(5)
        clear_output(wait=True)
except KeyboardInterrupt:
    print("Interrupted by user — stopping streams...")
    
except Exception as e:
    print("Streaming failed:", e)
    
# from pyspark.sql.functions import to_json, struct

# for x in range(0, 2000):
#     try:
#         print("Showing live view refreshed every 5 seconds")
#         print(f"Seconds passed: {x*5}")
        
#         predictedDf = parsedDF.withColumn("predicted_label", predict_batch_udf(col("comment")))
#         # # 1) bring to pandas
#         # pdf = batchDF.toPandas()

#         # # 2) model inference in pandas
#         # pdf["predicted_label"] = pdf["comment"].apply(predict_top_label)

#         # 3) display
#         display(predictedDf.toPandas())
        
#         sleep(5)
#         clear_output(wait=True)
#     except KeyboardInterrupt:
#         print("break")
#         break

# print("Live view ended...")

# for x in range(0, 2000):
#     try:
#         print("Showing live view refreshed every 5 seconds")
#         print(f"Seconds passed: {x*5}")
#         result1 = spark.sql(f"SELECT * from predictions")
#         display(result1.toPandas())
#         sleep(5)
#         clear_output(wait=True)
#     except KeyboardInterrupt:
#         print("break")
#         break



+-------+
|comment|
+-------+
+-------+



None